In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *

!pip install catboost
from catboost import CatBoostClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 66.1MB 42kB/s 


In [2]:
import nltk
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 71kB 2.2MB/s 
     |████████████████████████████████| 890kB 7.3MB/s 
     |████████████████████████████████| 890kB 31.5MB/s 
     |████████████████████████████████| 1.1MB 37.5MB/s 
     |████████████████████████████████| 3.0MB 39.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp36-none-any.whl size=101182 sha256=91de38ba5661cfc23c9acce6f17bf3b93f16c3ceaf809589e60db42fb93b3d57
  Stored in directory: /root/.cache/pip/wheels/6f/3f/75/c0c4b3ef5dfbf8806d37b8dc661861772aba2f7aa419c85a9b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f48839c9339995e5d5ecb4f30d0f72f29db78fa6dbbf124e68170894adb893e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [4]:
full_df = pd.concat([train, test]).reset_index()
full_df.drop('index',axis=1,inplace=True)
full_df.drop('Text_Tag',axis=1,inplace=True)
full_df


,Labels,Text
0,1.0,Says the Annies List political group supports ...
1,2.0,When did the decline of coal start? It started...
2,3.0,"Hillary Clinton agrees with John McCain ""by vo..."
3,1.0,Health care reform legislation is likely to ma...
4,2.0,The economic turnaround started at the end of ...
...,...,...
11502,NaN,Says his budget provides the highest state fun...
11503,NaN,Ive been here almost every day.
11504,NaN,"In the early 1980s, Sen. Edward Kennedy secret..."
11505,NaN,Says an EPA permit languished under Strickland...


In [5]:
word_embeddings = SentenceTransformer('roberta-large-nli-stsb-mean-tokens').encode(full_df.Text)

100%|██████████| 1.31G/1.31G [02:00<00:00, 10.9MB/s]


In [6]:
word_embeddings = pd.DataFrame(data=word_embeddings)
word_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.843113,0.354266,0.356839,0.419612,0.502846,0.265881,1.124245,-0.337539,-1.011597,0.389831,-1.143914,-1.498768,0.349013,0.630598,-0.308929,-1.211497,0.639380,-0.070070,0.197310,-0.694955,-0.748948,-0.985393,-1.381263,-0.965011,0.196261,-0.165667,-0.192807,-0.399925,-0.571577,-0.015549,0.400864,-0.732438,0.259238,0.038886,-1.680591,-0.752563,0.343458,-0.785571,-1.548424,-0.629427,...,-0.416050,-0.434246,-0.105925,0.207332,-0.963040,-0.041007,-0.155754,0.277735,0.030981,-0.246306,0.552735,-0.799284,-0.250765,-1.899152,-0.518160,0.620628,0.464231,-0.309856,0.085772,0.445845,0.001798,0.197886,1.256550,-0.412361,-0.562702,-1.153980,0.355673,-0.240783,-1.033721,1.400332,0.022141,0.510811,0.853409,0.351012,0.043098,0.612751,-0.362309,0.728115,-0.631633,-0.265538
1,-0.154687,0.872684,1.772497,-0.295066,0.222306,-0.253726,-1.300731,0.190504,0.343304,-0.872295,0.883556,-0.409843,-0.357488,-0.139657,0.384983,-0.794849,-1.066065,-0.051639,0.139327,0.139749,0.117898,0.461543,-1.046779,0.199070,-1.169691,-0.055779,-0.828227,-1.178741,0.154195,-1.155196,0.319821,-0.137026,-0.893601,-0.493398,0.772720,-0.778966,-0.642559,-0.128265,1.243242,-1.194343,...,0.877195,0.133658,-0.019471,-0.440079,0.622855,-0.382756,-0.269157,-0.771482,-0.626759,-1.214525,1.213236,-0.706025,-0.896191,-0.587146,0.188100,0.173347,-1.696494,-0.575991,-0.401825,-0.878120,0.333844,-0.645788,-0.933316,0.095632,-1.189360,-0.747389,-0.634917,1.291249,-1.595213,0.180152,-0.493622,0.729952,-0.513198,1.092363,-0.258281,0.267083,-0.399990,-0.463839,-1.466221,0.220743
2,-1.090439,-1.039243,-0.442930,-1.469323,0.165641,-0.411887,-0.524987,1.435755,-0.221388,0.336802,-0.551851,-0.579335,-1.103295,1.014058,0.514499,0.020389,1.099066,1.208101,0.988841,1.010967,-0.034147,-1.322206,-1.929260,-0.313306,-0.597575,0.154907,0.451024,-0.220749,-0.292568,-0.569620,-1.047028,-1.128314,0.382467,1.066755,-0.906427,-0.978917,0.257975,-0.024066,-1.304759,0.307851,...,0.383584,-0.643162,-0.567528,0.672795,0.460016,1.943318,-0.099922,-0.545651,0.080036,-0.051010,0.285562,0.600908,-0.681900,1.709387,-0.631731,0.168582,-0.115423,0.145797,-1.153039,0.255021,-1.061557,0.622468,0.076881,-1.268889,0.564817,0.068600,-0.259890,0.401971,-0.329564,1.528328,0.896938,1.188235,-0.614642,0.053428,-0.562576,0.325780,-1.103372,-1.954507,0.481241,-0.146407
3,0.599440,0.623602,0.633819,0.405363,0.706752,-0.079211,0.344765,-0.220469,-0.569685,-0.079065,-0.076758,1.569412,-0.278345,0.464309,-0.433065,-1.224822,-0.210659,-0.190351,-0.811026,-1.186909,-0.193128,0.878078,1.618985,-0.496415,1.310311,-0.229879,-0.483367,-0.503116,0.748487,-0.606732,-1.408139,-0.482079,-0.711370,0.238348,-0.895124,0.667424,-0.624264,-0.758625,-0.358398,-0.065462,...,-0.312904,-0.635221,-0.146240,-0.417504,-0.173062,0.587925,0.703627,0.999841,-0.951558,-0.135231,0.494266,-0.434182,-1.080807,-0.713855,-0.633628,0.060904,0.127755,-0.921143,-0.846210,-0.678195,-0.489322,-1.591884,0.925410,-0.127750,0.404707,-0.285969,0.945852,-0.417142,-1.571512,1.861141,0.311482,0.002135,-0.285264,0.073709,0.787635,-0.344154,-0.042826,1.363384,0.595176,-0.336081
4,-0.092458,-0.040650,-0.215230,0.439569,1.114173,0.060738,-0.115374,-0.424731,0.706269,-0.155483,0.662838,0.791877,-0.067191,-0.313515,-0.166139,0.178110,-0.376365,0.253624,-1.166783,-1.275676,0.633950,0.408043,-1.205060,-1.429486,-0.148764,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632213,1.001485,0.441519,0.425402,-0.661800,2.639185,0.492243,0.714778,0.791690,-0.303791,...,0.680721,1.496218,-0.320827,0.126428,0.195708,0.626634,-0.573826,0.262326,-0.785340,-0.277522,2.571322,1.039393,-0.520889,-1.399410,1.457769,0.471625,-2.558427,-0.643707,-0.414140,0.643783,0.277404,-2.057263,-0.369222,0.34327

In [7]:
from sklearn.decomposition import PCA
pca = pd.DataFrame(data=PCA(10).fit_transform(word_embeddings), columns=[f'pca{i}' for i in range(1,11)])

In [8]:
full_df['Length'] = full_df['Text'].apply(len)


In [9]:
full_df = pd.concat([full_df.drop('Text',axis=1),word_embeddings,pca],axis=1)
full_df.Labels

0        1.0
1        2.0
2        3.0
3        1.0
4        2.0
        ... 
11502    NaN
11503    NaN
11504    NaN
11505    NaN
11506    NaN
Name: Labels, Length: 11507, dtype: float64

In [10]:
train_df = full_df[full_df.Labels.notnull()]
test_df = full_df[full_df.Labels.isnull()]

In [11]:
test_df.shape

(1267, 1036)

In [12]:
test.shape

(1267, 2)

In [13]:
train_df.shape,train.shape

((10240, 1036), (10240, 3))

In [14]:
X, y = train_df.drop('Labels', axis=1), train_df.Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train, y_train.astype(int),
              eval_set=(X_test, y_test.astype(int)),
              early_stopping_rounds=100,
              )

Learning rate set to 0.047543
0:	learn: 1.7862223	test: 1.7878029	best: 1.7878029 (0)	total: 76.9ms	remaining: 12m 48s
1:	learn: 1.7806779	test: 1.7837998	best: 1.7837998 (1)	total: 144ms	remaining: 12m 1s
2:	learn: 1.7752954	test: 1.7801253	best: 1.7801253 (2)	total: 216ms	remaining: 11m 58s
3:	learn: 1.7701875	test: 1.7768494	best: 1.7768494 (3)	total: 282ms	remaining: 11m 44s
4:	learn: 1.7653781	test: 1.7736959	best: 1.7736959 (4)	total: 342ms	remaining: 11m 24s
5:	learn: 1.7610658	test: 1.7704377	best: 1.7704377 (5)	total: 401ms	remaining: 11m 7s
6:	learn: 1.7573758	test: 1.7675637	best: 1.7675637 (6)	total: 452ms	remaining: 10m 45s
7:	learn: 1.7533957	test: 1.7651081	best: 1.7651081 (7)	total: 513ms	remaining: 10m 40s
8:	learn: 1.7491463	test: 1.7628031	best: 1.7628031 (8)	total: 570ms	remaining: 10m 33s
9:	learn: 1.7449816	test: 1.7600610	best: 1.7600610 (9)	total: 622ms	remaining: 10m 21s
10:	learn: 1.7414482	test: 1.7579076	best: 1.7579076 (10)	total: 683ms	remaining: 10m 20s
1

In [16]:
importance = pd.DataFrame(data=model_cat.feature_importances_, index=X_train.columns, columns=['imp']).sort_values(by='imp',ascending=False)
imp_feat = importance[importance['imp'] > 0.05].index
imp_feat

Index([     987,   'pca2', 'Length',      747,      670,      360,      218,
             32,      597,      399,
       ...
            797,      921,      461,     1014,      345,      381,      835,
            873,      178,      309],
      dtype='object', length=642)

In [17]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train[imp_feat], y_train.astype(int),
              eval_set=(X_test[imp_feat], y_test.astype(int)),
              early_stopping_rounds=100,
              )

Learning rate set to 0.047543
0:	learn: 1.7859229	test: 1.7875806	best: 1.7875806 (0)	total: 38.7ms	remaining: 6m 26s
1:	learn: 1.7812357	test: 1.7844423	best: 1.7844423 (1)	total: 72.9ms	remaining: 6m 4s
2:	learn: 1.7758993	test: 1.7805254	best: 1.7805254 (2)	total: 115ms	remaining: 6m 22s
3:	learn: 1.7713463	test: 1.7774230	best: 1.7774230 (3)	total: 151ms	remaining: 6m 16s
4:	learn: 1.7667987	test: 1.7740165	best: 1.7740165 (4)	total: 185ms	remaining: 6m 10s
5:	learn: 1.7623332	test: 1.7711439	best: 1.7711439 (5)	total: 220ms	remaining: 6m 6s
6:	learn: 1.7581239	test: 1.7688104	best: 1.7688104 (6)	total: 258ms	remaining: 6m 8s
7:	learn: 1.7536680	test: 1.7661620	best: 1.7661620 (7)	total: 293ms	remaining: 6m 6s
8:	learn: 1.7499409	test: 1.7638698	best: 1.7638698 (8)	total: 328ms	remaining: 6m 3s
9:	learn: 1.7461996	test: 1.7614317	best: 1.7614317 (9)	total: 362ms	remaining: 6m 1s
10:	learn: 1.7422932	test: 1.7589309	best: 1.7589309 (10)	total: 397ms	remaining: 6m
11:	learn: 1.738797

In [20]:
prediction = model_cat.predict_proba(test_df[X_train.columns])
submission = pd.DataFrame(prediction, columns=[0,1,2,3,4,5])
submission.to_csv('submission_large.csv', index=False)

In [ ]:
submission

,0,1,2,3,4,5
0,0.147928,0.182078,0.218269,0.224169,0.066129,0.161427
1,0.158105,0.210149,0.272833,0.148255,0.057894,0.152765
2,0.154479,0.234471,0.127751,0.107683,0.199263,0.176353
3,0.284520,0.248342,0.223991,0.095079,0.072980,0.075088
4,0.224200,0.221353,0.185514,0.152297,0.091304,0.125333
...,...,...,...,...,...,...
1262,0.114394,0.177917,0.267689,0.289591,0.040772,0.109637
1263,0.123652,0.187271,0.159689,0.245893,0.056852,0.226643
1264,0.205273,0.274678,0.190959,0.118966,0.098632,0.111492
1265,0.151320,0.170056,0.215246,0.211572,0.076569,0.175236


In [22]:
sub1 = pd.read_csv('submission_large.csv')
sub2 = pd.read_csv('submission.csv')
final = (sub1+sub2)/2
final

,0,1,2,3,4,5
0,0.142635,0.141086,0.183992,0.272996,0.063123,0.196168
1,0.123357,0.194093,0.260181,0.237883,0.061257,0.123229
2,0.136626,0.243739,0.208981,0.139872,0.100317,0.170465
3,0.261637,0.244041,0.186312,0.125054,0.089388,0.093568
4,0.132027,0.268715,0.218802,0.160557,0.106047,0.113853
...,...,...,...,...,...,...
1262,0.125971,0.130865,0.212088,0.302857,0.028259,0.199961
1263,0.128214,0.179892,0.144849,0.215654,0.057469,0.273922
1264,0.173911,0.290898,0.187895,0.138457,0.110619,0.098220
1265,0.128142,0.213282,0.212288,0.219637,0.065762,0.160888


In [23]:
final.to_csv('final.csv',index=False)
